A compact, readable Python implementation of ELIZA (DOCTOR script style).

This implements a simplified version of Weizenbaum's algorithm:
- A keyword list with ranking
- Decomposition patterns using simple wildcards ("*" for any words)
- Reassembly templates that can include references to captured groups like {1}, {2}
- On-the-fly pronoun swaps (I -> you, my -> your, etc.)


The script below contains a small DOCTOR-like script sufficient for demo.


This file is intended for learning and tinkering; it's not a perfect re-creation
of the original MAD-SLIP ELIZA, but follows the same core ideas.

In [34]:
import re
import random
import sys

In [35]:
PRONOUN_MAP = {
    # first-person -> second-person
    "i": "you",
    "me": "you",
    "my": "your",
    "mine": "yours",
    "am": "are",
    # second-person -> first-person
    "you": "I",
    "your": "my",
    "yours": "mine",
    # past tense
    "was": "were",
    "were": "was",
    # common contractions (simple handling)
    "i'm": "you're",
    "im": "you're",
    "you're": "I'm",
}


In [36]:
def reflect(fragment: str) -> str:
    """Swap first/second-person pronouns in a fragment, preserving punctuation and capitalization.
    Heuristic verb agreement: map 'are' <-> 'am' only when the subject is a pronoun that should change.
    """
    if not fragment:
        return fragment
    tokens = re.findall(r"\w+|[^\w\s]+", fragment)
    out_tokens = []
    # helper to get previous word token lowercased
    def prev_word(tokens, idx):
        for j in range(idx-1, -1, -1):
            if re.match(r"\w+", tokens[j]):
                return tokens[j].lower()
        return None

    for i, t in enumerate(tokens):
        if re.match(r"\w+", t):
            key = re.sub(r"[^a-z']", '', t.lower())
            # special-case verb agreement
            if key in ('are', 'am', 'was', 'were'):
                pw = prev_word(tokens, i)
                if pw in ("you", "you're", "youre"):
                    # if original subject is 'you', use first-person verb form
                    if key == 'are':
                        repl = 'am'
                    elif key == 'was':
                        repl = 'was'  # 'you was' -> 'I was' (leave as was)
                    else:
                        repl = key
                    out_tokens.append(repl)
                    continue
                if pw in ("i", "i'm", "im"):
                    # original subject is I -> second-person verb
                    if key == 'am':
                        repl = 'are'
                    elif key == 'was':
                        repl = 'were'
                    else:
                        repl = key
                    out_tokens.append(repl)
                    continue
                # otherwise fall through to normal mapping (do not flip)
            repl = PRONOUN_MAP.get(key)
            out_tokens.append(repl if repl else t)
        else:
            out_tokens.append(t)
    # join tokens and fix spacing before punctuation
    s = ' '.join(out_tokens)
    s = re.sub(r'\s+([.,!?;:])', r'\1', s)
    s = s.strip()
    # Capitalize first character of the whole fragment only
    if s:
        s = s[0].upper() + s[1:]
    return s


In [37]:
# ---------- Script format ----------
SCRIPT = {
	'you': {
		'rank': 4,
		'decomps': [
			("* you are *", [
				"What makes you think I am {1}?",
				"Does it please you to believe I am {1}?",
			]),
			("* you * me *", [
				"What makes you think I {1} you {2}?",
				"Why do you say I {1} you {2}?",
			])
		]
	},
	'mother': {
		'rank': 3,
		'decomps': [
			("* my mother *", [
				"Tell me more about your mother.",
				"What was your relationship with your mother like?",
			])
		]
	},
	'father': {
		'rank': 3,
		'decomps': [
			("* my father *", [
				"How did your father influence you?",
				"Tell me more about your father.",
			])
		]
	},
	'dream': {
		'rank': 2,
		'decomps': [
			("* dream *", [
				"What does that dream suggest to you?",
				"Have you had similar dreams before?",
			])
		]
	},
	'men': {
		'rank': 1,
		'decomps': [
			("* men are all alike *", [
				"IN WHAT WAY?",
				"Can you give a specific example?",
			])
		]
	},
	# generic fallback - lowest priority
	'null': {
		'rank': 0,
		'decomps': [
			("*", [
				"Please go on.",
				"Can you say more about that?",
				"Do you often feel that way?",
			])
		]
	}
}

In [38]:
# ---------- Pattern matching helpers ----------

def match_decomposition(pattern, text):
	"""
	Match a decomposition pattern like "* you are *" to the input text.
	Returns a list of captured groups, or None if no match.
	This version normalizes whitespace so that multiple spaces still match.
	It also allows the trailing/leading wildcard to be optional when pattern starts/ends with '*'.
	"""
	# normalize whitespace in text and pattern
	text_norm = re.sub(r'\s+', ' ', text.strip())
	# build regex from pattern allowing optional leading/trailing groups
	escaped = re.escape(pattern).replace(r'\*', '(.*)')
	# make spaces flexible
	escaped = escaped.replace(' ', '\\s+')
	# allow leading/trailing wildcard to match empty string by making (.*) non-greedy
	escaped = escaped.replace('(.*)', '(.*?)')
	pat = '^' + escaped + '$'
	m = re.match(pat, text_norm, re.IGNORECASE)
	if m:
		# strip and normalize groups
		return [re.sub(r'\s+', ' ', g.strip()) for g in m.groups()]
	return None


class Eliza:
	def __init__(self, script):
		self.script = script
		self.history = []  # store (user, reply) pairs

	def find_keywords(self, user_input):
		"""Return a list of keywords found in the user input, ordered by rank (desc) and script order."""
		found = []
		lowered = ' ' + re.sub(r'\s+', ' ', user_input.lower()) + ' '
		for k, v in self.script.items():
			if k == 'null':
				continue
			# match whole words only (avoid substrings)
			if re.search(r'\b' + re.escape(k) + r'\b', lowered):
				found.append((k, v['rank']))
		# stable sort by rank desc while preserving script ordering for ties
		found.sort(key=lambda kv: -kv[1])
		return [k for k, _ in found]

	def respond(self, user_input: str) -> str:
		user_input_clean = user_input.strip()
		# If input is empty, prompt
		if not user_input_clean:
			return "Please say something."

		# Primary pass: scan all keys in rank order and test decompositions directly.
		keys_by_rank = sorted([k for k in self.script.keys() if k != 'null'],
					key=lambda k: -self.script[k].get('rank', 0))

		for key in keys_by_rank:
			entry = self.script.get(key)
			if not entry:
				continue
			for pattern, reassemblies in entry.get('decomps', []):
				groups = match_decomposition(pattern, user_input_clean)
				if groups is None:
					continue
				# We have a match. Choose a reassembly
				template = random.choice(reassemblies)
				# Replace placeholders {1}, {2}, ... with reflected groups
				def repl_placeholder(match):
					idx = int(match.group(1))
					if idx <= len(groups):
						frag = groups[idx - 1]
						return reflect(frag)
					return ''
				response = re.sub(r'\{(\d+)\}', repl_placeholder, template)
				# If no placeholders used, still try to do small pronoun reflection for nicer replies
				if '{' not in template:
					# append the first captured fragment after reflection (if any)
					if groups and groups[0]:
						# make appended fragment lowercase to fit mid-sentence templates
						frag = reflect(groups[0])
						if frag:
							frag = frag[0].lower() + frag[1:]
							response = response + ' ' + frag
				# record history and return
				self.history.append((user_input_clean, response))
				return response

		# Fallback: try 'null' patterns
		null_entry = self.script.get('null')
		if null_entry:
			for pattern, reassemblies in null_entry.get('decomps', []):
				groups = match_decomposition(pattern, user_input_clean)
				if groups is None:
					continue
				template = random.choice(reassemblies)
				def repl_placeholder(match):
					idx = int(match.group(1))
					if idx <= len(groups):
						frag = groups[idx - 1]
						return reflect(frag)
					return ''
				response = re.sub(r'\{(\d+)\}', repl_placeholder, template)
				if '{' not in template:
					if groups and groups[0]:
						frag = reflect(groups[0])
						if frag:
							frag = frag[0].lower() + frag[1:]
							response = response + ' ' + frag
				self.history.append((user_input_clean, response))
				return response

		# Shouldn't reach here because of null fallback
		return "I don't know what to say."

	def save_history(self, path):
		with open(path, 'w', encoding='utf-8') as f:
			for u, r in self.history:
				f.write(f'User: {u}\nEliza: {r}\n')


In [39]:
# --- Demo: non-interactive example conversation ---
demo_eliza = Eliza(SCRIPT)
examples = [
    'I feel sad about my mother',
    'You are not listening to me',
    'I had a strange dream last night',
    'Sometimes I think men are all alike',
]
for e in examples:
    print('User:', e)
    print('Eliza:', demo_eliza.respond(e))


User: I feel sad about my mother
Eliza: Can you say more about that? you feel sad about your mother
User: You are not listening to me
Eliza: Can you say more about that? i am not listening to you
User: I had a strange dream last night
Eliza: Please go on. you had a strange dream last night
User: Sometimes I think men are all alike
Eliza: Can you say more about that? sometimes you think men are all alike


In [40]:
# show collected history from demo
print('\nDemo conversation history:')
for u, r in demo_eliza.history:
    print(f'User: {u}\nEliza: {r}\n')


Demo conversation history:
User: I feel sad about my mother
Eliza: Can you say more about that? you feel sad about your mother

User: You are not listening to me
Eliza: Can you say more about that? i am not listening to you

User: I had a strange dream last night
Eliza: Please go on. you had a strange dream last night

User: Sometimes I think men are all alike
Eliza: Can you say more about that? sometimes you think men are all alike



In [41]:
# Interactive Session

print("ELIZA (Python) — type 'quit' or 'exit' to leave.\n")
eliza = Eliza(SCRIPT)
while True:
    try:
        user = input('> ')
    except (EOFError, KeyboardInterrupt):
        print('\nGoodbye.')
        break
    if not user:
        continue
    if user.lower() in ('quit', 'exit', 'bye'):
        print('Goodbye. Take care!')
        break
    reply = eliza.respond(user)
    print(reply)

ELIZA (Python) — type 'quit' or 'exit' to leave.

Goodbye. Take care!
Goodbye. Take care!
